importing required libraries

In [1]:
import os
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfTransformer
from appJar import gui
import numpy as np

importing train data and also stop words

In [2]:
os.chdir("C:/Users/karthi/Desktop/sentiment")
with open('imdb.txt','r') as file:
    data=file.readlines()
stop_words=stopwords.words('english')

stop words are words that is generally occurs in english language like is,he,she,etc.,
These words removed from train data during preprocessing

In [4]:
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

this will clean the data by removing stop words, tags and other unwanted symbols

In [ ]:
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = (re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-', ''))
    text=text.split(' ')
    for each in text:
        chk=[True for s in stop_words if s==each]
        if chk==[True]:
            text.remove(each)
    text=' '.join(text[0:len(text)-2])
    return text

creating the bag of words 

In [6]:
train_x,train_y=[],[]
clean_data=[ preprocessor(x).rsplit(' ',1) for x in data]
print(clean_data)
for each in clean_data:
    train_x.append(each[0])
    train_y.append(int(each[1]))
count = CountVectorizer()
word_bag = count.fit_transform(train_x)


[['very very slow moving aimless movie a distressed drifting young man', '0'], ['sure was lost flat characters the audience nearly half whom walked', '0'], ['attempting artiness black white clever camera angles movie disappointed became even ridiculous the acting poor the plot lines almost non existent', '0'], ['little music anything speak', '0'], ['best scene the movie when gerardo trying find song keeps running his head', '1'], ['rest the movie lacks art charm meaning about emptiness it works guess it empty', '0'], ['wasted two hours', '0'], ['saw movie today thought was good effort good messages kids', '1'], ['bit predictable', '0'], ['loved casting jimmy buffet the science teacher', '1'], ['those baby owls adorable', '1'], ['movie showed lot florida best made it look appealing', '1'], ['songs the best the muppets so hilarious', '1'], ['was cool', '1'], ['is very right case movie delivers everything almost right your face', '1'], ['had average acting the main person it a low budget 

Indexing based on the frequency of the words

In [7]:
tfidf = TfidfTransformer(use_idf=True,norm='l2',smooth_idf=True)
np.set_printoptions(precision=2)
train_x=tfidf.fit_transform(word_bag).toarray()

C:\Users\karthi\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


Fitting the data 

In [8]:
cls=LogisticRegression()
cls.fit(train_x,train_y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

predict function

In [9]:
def predict(sent):
    test=np.zeros(3030)
    pos=[]
    for each in preprocessor(sent).split(' '):
        pos.append(count.vocabulary_.get(each,4000))
    for each in pos:
        if each<3031:
            test[each]=test[each]+1
    return cls.predict(test)  

GUI interface

In [12]:
def check(k):
    value=app.getEntry('review')
    value=predict(value)
    if value==[1]:
        app.setImage("profpic","love.png")
    elif value==[0]:
        app.setImage("profpic","angry.png")
    else:
        pass
app=gui("sentiment")
app.setFont(size=25)
app.setSize(600,500)
app.setResizable(True)
app.addLabel("right","Review:",1,0)
app.addEntry("review",1,1)
app.addLabel("left","       ",1,2)
app.addButtons(['check'],check,2,1)
app.addImage("profpic","neutral.png",3,1)
app.go()

C:\Users\karthi\Anaconda3\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C:\Users\karthi\Anaconda3\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
